<a href="https://colab.research.google.com/github/Martinmbiro/Card-classification/blob/main/01%20Cards%20modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementing Modules**
> In this notebook, I'll be creating modules for the end to end project on card classification

> 💎 **Pro Tip**
+ Modules help organize code logically, promote code reusability and cleaner code

In [ ]:
# to delete modules.zip and helper_modules folder
# !rm -rf /content/helper_modules/
# !rm -rf /content/modules.zip

In [ ]:
# import torch, torchvision, pathlib
import torch, torchvision, pathlib

# print versions
print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')

torch version: 2.6.0+cu124
torchvision version: 0.21.0+cu124


> 📝 **Note**  
+ To **write** a code cell's content into a `*.py`, file we'll use the _magic command_ `%%writefile filename.py`
+ To **append** a code cell's content into a `*.py`, file we'll use the _magic command_ `%%writefile -a filename.py`

### Load the data
> First, we'll create a directory to hold all the custom modules we write
+ To create directories, we'll make use of the [`pathlib`](https://docs.python.org/3/library/pathlib.html) python module

In [ ]:
# create directory for helper modules
HELPER_MODULES = pathlib.Path('helper_modules')
HELPER_MODULES.mkdir(parents=True, exist_ok=True)

In [ ]:
%%writefile helper_modules/data_loader.py
import kagglehub as kh, zipfile, shutil, os, pathlib, torch, numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms.v2 as T

# Dataset Wrapper class
class CardsWrapper(Dataset):
  def __init__(self, path:pathlib.PosixPath, transform:T.Compose):
    self.d_set = ImageFolder(root=path, transform=transform)

  def __len__(self):
    return len(self.d_set)

  def __getitem__(self, index):
    img, lb = self.d_set[index]
    return img, lb

  @property
  def idx_to_class(self):
    mapper = {idx : cls for cls, idx in self.d_set.class_to_idx.items()}
    return mapper

  @property
  def classes(self):
    return self.d_set.classes

# define image transforms
_card_transforms = T.Compose([
    T.PILToTensor(),
    T.ToDtype(torch.float32, scale=True),
    T.Resize((128,128)),
    T.CenterCrop((128, 128))
])

# download folder
_DOWNLOAD_DIR = pathlib.Path('cards')
_DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

# function to return dataloaders for train, validation, test
def get_dataloaders() -> tuple[DataLoader, DataLoader, DataLoader, dict[int, str]]:
  """
    Downloads, extracts, and creates dataloaders for the cards image dataset.

    This function downloads the "cards-image-datasetclassification" dataset from Kaggle,
    archives it, extracts it to a local directory, and then creates PyTorch DataLoader
    objects for the training, testing, and validation sets.

    Returns
    -------
    tuple
        A tuple containing the following elements:
            - train_dl (DataLoader): DataLoader for the training set.
            - test_dl (DataLoader): DataLoader for the testing set.
            - val_dl (DataLoader): DataLoader for the validation set.
            - idx_to_class (dict[int, str]): A dictionary mapping class indices to class labels.

    Raises
    ------
    FileNotFoundError
        If the Kaggle dataset download fails or the zip file cannot be found.
    Exception
        If any other errors occur during dataset processing.

    Example
    -------
    >>> train_loader, test_loader, val_loader, class_mapping = get_dataloaders()
    >>> print(type(train_loader))
    <class 'torch.utils.data.dataloader.DataLoader'>
    >>> print(type(class_mapping))
    <class 'dict'>
  """
  # Download latest version of dataset from kaggle
  _cache = kh.dataset_download("gpiosenka/cards-image-datasetclassification")

  # archive the files
  shutil.make_archive(base_name=_DOWNLOAD_DIR/'data', format='zip', root_dir=_cache)

  # extract the files
  with zipfile.ZipFile(_DOWNLOAD_DIR/'data.zip', mode='r') as zipf:
    zipf.extractall(path=_DOWNLOAD_DIR)

  # create datasets first
  train_data = CardsWrapper(path='/content/cards/train', transform=_card_transforms)
  test_data = CardsWrapper(path='/content/cards/test', transform=_card_transforms)
  val_data = CardsWrapper(path='/content/cards/valid', transform=_card_transforms)

  # create dataloaders
  train_dl = DataLoader(
      dataset=train_data, batch_size=32, shuffle=True, pin_memory=True, num_workers=os.cpu_count())
  test_dl = DataLoader(
      dataset=test_data, batch_size=32, shuffle=True, pin_memory=True, num_workers=os.cpu_count())
  val_dl = DataLoader(
      dataset=val_data, batch_size=32, shuffle=True, pin_memory=True, num_workers=os.cpu_count())

  # return
  return train_dl, test_dl, val_dl, train_data.idx_to_class

Writing helper_modules/data_loader.py


## Build the [`resnet`](https://research.google/blog/efficientnet-improving-accuracy-and-efficiency-through-automl-and-model-scaling/#:~:text=EfficientNet%2DB0%20is%20the%20baseline,than%20the%20best%20existing%20CNN.) architecture
> With the help of the [`timm`](https://huggingface.co/docs/timm/v1.0.15/en/index) library, I'll load the [`resnet14t`](https://huggingface.co/timm/resnet14t.c3_in1k) CNN architecture and alter the `classifier` layer by specifying `53` classes

> The function defined here will return a `model`, `Optimizer` and `loss function`
+ Also, we'll use [`torch.optim.Adam`](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#adam) as optimizer and [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#crossentropyloss) as loss function, since this is a multi-class classification problem


> 🔔 **Info**
+ A standard deck of cards contains `52` cards from _clubs_, _hearts_, _diamonds_ and _spades_ suits, and a _joker_ card
+ A quickstart guide to using `timm` is linked [here](https://huggingface.co/docs/timm/v1.0.15/en/quickstart#quickstart)
+ For better generalization and to reduce training time, we'll leverage on **Transfer Learning.** Hence, the model we declare here will be **pre-trained** from the start

In [ ]:
%%writefile helper_modules/model_builder.py
import timm, torch

def get_model(device:str) -> tuple[torch.nn.Module, torch.optim.Optimizer, torch.nn.Module]:
  """
    Creates and initializes a model, optimizer, and loss function for training.

    Parameters
    ----------
    device : str
        The device to which the model should be moved. Common values include 'cuda' for GPU or 'cpu' for CPU.

    Returns
    -------
    tuple
        A tuple containing three elements:
        - model (torch.nn.Module): The model initialized with the resnet14t architecture for 53 classes.
        - opt (torch.optim.Optimizer): The Adam optimizer initialized with the model's parameters.
        - loss_fn (torch.nn.Module): The CrossEntropyLoss function used for multi-class classification.
    """
  # pretrained model, with 53 classes for output layer
  model = timm.create_model(
      model_name='resnet14t',
      num_classes=53,
      pretrained=True).to(device)

  # optimizer
  opt = torch.optim.Adam(params=model.parameters(), lr=0.001)

  # loss function
  loss_fn = torch.nn.CrossEntropyLoss()

  return model, opt, loss_fn

Writing helper_modules/model_builder.py


## Early stopping
> 💎 **Pro Tip**

> [Early stopping](https://www.linkedin.com/advice/1/what-benefits-drawbacks-early-stopping#:~:text=Early%20stopping%20is%20a%20form,to%20increase%20or%20stops%20improving.) is a mechanism of stopping training when the validation loss stops improving; with a view to preventing _overfitting_ on the training data
+ Here, we'll create a class to take care of _early-stopping_

In [ ]:
%%writefile helper_modules/utils.py
import torch, pathlib, numpy as np, matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import ConfusionMatrixDisplay
from torch.utils.data import Dataset, Subset
import torch, seaborn as sns, pandas as pd
from copy import deepcopy
from google.colab import files
from PIL import Image
import torchvision.transforms.v2 as T
from pathlib import Path
from itertools import chain
from matplotlib.gridspec import GridSpec

class EarlyStopping:
  """
  Early stopping to prevent overfitting.

  Attributes
  ----------
  counter : int
      Counter to track the number of epochs without improvement.
  patience : int
      Number of epochs to wait after the last best score.
  min_delta : float
      Minimum change in the monitored quantity to qualify as an improvement.
  score_type : str
      'loss' or 'metric', determines the direction of improvement.
  best_epoch : int
      Epoch with the best score.
  best_score : float
      Best score achieved so far.
  best_state_dict : dict
      State dictionary of the model at the best score.
  stop_early : bool
      Flag to indicate if early stopping should be triggered.
  """

  def __init__(self, score_type: str, min_delta: float = 0.0, patience: int = 5):
    """
    Initializes the EarlyStopping object.

    Parameters
    ----------
    score_type : str
        'loss' or 'metric', determines the direction of improvement.
    min_delta : float, optional
        Minimum change in the monitored quantity to qualify as an improvement. Defaults to 0.0.
    patience : int, optional
        Number of epochs to wait after the last best score. Defaults to 5.

    Raises
    ------
    Exception
        If score_type is not 'metric' or 'loss'.
    """
    self.counter = 0
    self.patience = patience
    self.min_delta = min_delta
    self.score_type = score_type
    self.best_epoch = None
    self.best_score = None
    self.best_state_dict = None
    self.stop_early = False

    if (self.score_type != 'metric') and (self.score_type != 'loss'):
        err_msg = 'score_type can only be "metric" or "loss"'
        raise Exception(err_msg)

  def __call__(self, model: torch.nn.Module, ep: int, ts_score: float):
    """
    Checks if early stopping should be triggered based on the current score.

    Parameters
    ----------
    model : torch.nn.Module
        The model being trained.
    ep : int
        The current epoch number.
    ts_score : float
        The current score (loss or metric).
    """
    if self.best_epoch is None:
        self.best_epoch = ep
        self.best_score = ts_score
        self.best_state_dict = deepcopy(model.state_dict())

    elif (self.best_score - ts_score >= self.min_delta) and (self.score_type == 'loss'):
        self.best_epoch = ep
        self.best_score = ts_score
        self.best_state_dict = deepcopy(model.state_dict())
        self.counter = 0

    elif (ts_score - self.best_score >= self.min_delta) and (self.score_type == 'metric'):
        self.best_epoch = ep
        self.best_score = ts_score
        self.best_state_dict = deepcopy(model.state_dict())
        self.counter = 0

    else:
        self.counter += 1
        if self.counter >= self.patience:
            self.stop_early = True


Writing helper_modules/utils.py


## Model training / evaluation
> Here, I'll define functions for training and testing batches of data, as well as a function to return true labels, `y_true`, prediction labels, `y_pred` and prediction probabilities `y_proba`

In [ ]:
%%writefile helper_modules/train_test.py
import torch, numpy as np
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score

# del torch, F, f1_score, accuracy_score

# function for model training
def train_batches(model: torch.nn.Module, train_dl: torch.utils.data.DataLoader,
                  optimizer: torch.optim.Optimizer, loss_fn: torch.nn.Module, device: str) -> tuple[float, float, float]:
  """
  Trains model on all batches of the training set DataLoader and returns
  average training loss, accuracy, and F1 score.

  Parameters
  ----------
  model : torch.nn.Module
      The model being trained.
  train_dl : torch.utils.data.DataLoader
      DataLoader for training data.
  optimizer : torch.optim.Optimizer
      The optimizer.
  loss_fn : torch.nn.Module
      Function used to calculate loss.
  device : str
      The device on which computation occurs.

  Returns
  -------
  tuple
      A tuple containing:
          - ls (float): Average training loss across all batches.
          - acc (float): Average training accuracy across all batches.
          - f1 (float): Average training F1 score across all batches.
  """
  # for reproducibility
  torch.manual_seed(0)
  torch.cuda.manual_seed(0)
  ls, acc, f1 = 0, 0, 0

  # training mode
  model.train()

  for x, y in train_dl:
      # move x, y to device
      x, y = x.to(device), y.to(device)
      # zero_grad
      optimizer.zero_grad()

      # forward pass
      logits = model(x)
      y_pred = F.softmax(logits, dim=1).argmax(dim=1).cpu().numpy()

      # loss
      loss = loss_fn(logits, y)
      # accumulate values
      ls += loss.item()
      acc += accuracy_score(y_true=y.cpu().numpy(), y_pred=y_pred)
      f1 += f1_score(y_true=y.cpu().numpy(),
                     y_pred=y_pred,
                     average='macro')

      # back propagation
      loss.backward()
      # optimizer step
      optimizer.step()

  # compute averages
  ls /= len(train_dl)
  acc /= len(train_dl)
  f1 /= len(train_dl)

  # return values
  return ls, acc, f1

def test_batches(model: torch.nn.Module, val_dl: torch.utils.data.DataLoader,
                 loss_fn: torch.nn.Module, device: str) -> tuple[float, float, float]:
  """
  Evaluates model on all batches of the test set DataLoader and returns
  average test loss, accuracy, and F1 score.

  Parameters
  ----------
  model : torch.nn.Module
      The model being evaluated.
  test_dl : torch.utils.data.DataLoader
      DataLoader for test data.
  loss_fn : torch.nn.Module
      Function used to calculate loss.
  device : str
      The device on which computation occurs.

  Returns
  -------
  tuple
      A tuple containing:
          - ls (float): Average test loss across all batches.
          - acc (float): Average test accuracy across all batches.
          - f1 (float): Average test F1 score across all batches.
  """
  ls, f1, acc = 0, 0, 0

  # evaluation-mode
  model.eval()

  with torch.inference_mode():
    for x, y in val_dl:
        # move x, y to device
        x, y = x.to(device), y.to(device)

        # forward pass
        logits = model(x)
        y_pred = F.softmax(logits, dim=1).argmax(dim=1).cpu().numpy()

        # loss
        loss = loss_fn(logits, y)

        # accumulate values
        ls += loss.item()
        acc += accuracy_score(y_true=y.cpu().numpy(), y_pred=y_pred)
        f1 += f1_score(y_true=y.cpu().numpy(),
                       y_pred=y_pred,
                       average='macro')

  # compute averages
  ls /= len(val_dl)
  acc /= len(val_dl)
  f1 /= len(val_dl)

  # return values
  return ls, acc, f1


def true_preds_proba(model: torch.nn.Module, test_dl: torch.utils.data.DataLoader,
                     device: str) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
  """
  A function that returns true labels, predictions, and prediction probabilities
  from the passed DataLoader.

  Parameters
  ----------
  model : torch.nn.Module
      A neural network that subclasses torch.nn.Module.
  test_dl : torch.utils.data.DataLoader
      A DataLoader for the test dataset.
  device : str
      The device on which computation occurs.

  Returns
  -------
  tuple
      A tuple containing:
          - y_true (np.ndarray): A numpy array with true labels.
          - y_pred (np.ndarray): A numpy array with predicted labels.
          - y_proba (np.ndarray): A numpy array with predicted probabilities.
  """
  # empty lists
  y_true, y_preds, y_proba = list(), list(), list()
  with torch.inference_mode():
      model.eval()  # set eval mode
      for x, y in test_dl:
          # move x to device
          x = x.to(device)

          # make prediction
          logits = model(x)

          # prediction and probabilities
          proba = F.softmax(logits, dim=1)
          pred = F.softmax(logits, dim=1).argmax(dim=1)

          # append
          y_preds.append(pred)
          y_proba.append(proba)
          y_true.append(y)

  y_preds = torch.concatenate(y_preds).cpu().numpy()
  y_proba = torch.concatenate(y_proba).cpu().numpy()
  y_true = torch.concatenate(y_true).numpy()

  return y_true, y_preds, y_proba


Writing helper_modules/train_test.py


## Plot results
> Here, I'll define helper functions for plotting training metrics

In [ ]:
%%writefile -a helper_modules/utils.py

# function to plot train and test results
def plot_train_results(ep_list: list, train_score: list, test_score: list,
                       ylabel: str, title: str, best_epoch: int):
  """
  Plots training and test results against each other.

  Parameters
  ----------
  ep_list : list
      A list containing all epochs used in the optimization loop.
  train_score : list
      A list containing the training scores from the optimization loop.
  test_score : list
      A list containing the test scores from the optimization loop.
  ylabel : str
      Label for the y-axis of the plot.
  title : str
      Title for the plot.
  best_epoch : int
      Best epoch for which early stopping occurred.

  Returns
  -------
  None
  """
  f, ax = plt.subplots(figsize=(5, 3), layout='constrained')

  # train loss
  ax.plot(ep_list, train_score, label='Training',
          linewidth=1.7, color='#0047ab')

  # test loss
  ax.plot(ep_list, test_score, label='Validation',
          linewidth=1.7, color='#990000')
  # vertical line (for early stopping)
  if best_epoch is not None:
      ax.axvline(best_epoch, linestyle='--', color='#000000', linewidth=1.0,
                  label=f'Best ep ({best_epoch})')

  # axis, title
  ax.set_title(title, weight='black')
  ax.set_ylabel(ylabel)
  ax.set_xlabel('Epoch')
  ax.tick_params(axis='both', labelsize=9)
  plt.grid(color='#e5e4e2')

  # legend
  f.legend(fontsize=9, loc='upper right',
            bbox_to_anchor=(1.28, 0.93),
            fancybox=False)

  plt.show()


def plot_confusion_matrix(y_true: np.ndarray, y_pred: np.ndarray):
  """
  Plots a confusion matrix for all classes.

  Parameters
  ----------
  y_true : np.ndarray
      An ndarray containing the true label values.
  y_pred : np.ndarray
      An ndarray containing the predicted label values.

  Returns
  -------
  None
  """
  # define figure and plot
  _, ax = plt.subplots(figsize=(8.0,8.0), layout='compressed')
  # plot
  ConfusionMatrixDisplay.from_predictions(
      y_true=y_true,
      y_pred=y_pred, cmap='Blues', colorbar=False, ax=ax)

  # set x and y labels
  ax.set_ylabel('True Labels', weight='black')
  ax.set_xlabel('Predicted Labels', weight='black',
                color='#dc143c')
  # set tick size and position
  ax.xaxis.tick_top()
  ax.xaxis.set_label_position('top')
  ax.tick_params(axis='both', labelsize=7)

  # change annotation font
  for txt in ax.texts:
      txt.set_fontsize(7.5)

  plt.show()


Appending to helper_modules/utils.py


## Save model
> 🔔 **Info**

> Pytorch's recommended way of saving a model is by saving its [`state_dict`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.state_dict). To do this, the [documentation](https://pytorch.org/tutorials/beginner/saving_loading_models.html#save-load-state-dict-recommended) recommends calling [`torch.save(obj=model.state_dict(), f=PATH)`](https://pytorch.org/docs/stable/generated/torch.save.html#torch-save)
+ `f` - a file-like object or a string or `os.PathLike` object containing a file name. To work with paths, we'll use Python's [`pathlib`](https://docs.python.org/3/library/pathlib.html) module
+ A common PyTorch convention is to save models using either a `.pt` or `.pth` file extension
+ Also, it's good practice to move the model to the `cpu` before saving its `state_dict`

In [ ]:
%%writefile -a helper_modules/utils.py

# function to save model to specified directory
def save_model(model: torch.nn.Module, path: pathlib.PosixPath):
    """
    Saves the model's state_dict to a specified path.

    Parameters
    ----------
    model : torch.nn.Module
        The model to save.
    path : pathlib.PosixPath
        The path where the model's state_dict will be saved.

    Returns
    -------
    None
    """
    torch.save(obj=model.cpu().state_dict(), f=path)
    print(f"MODEL'S state_dict SAVED TO: {path}")


Appending to helper_modules/utils.py


### Load saved model
> To load a previously saved model's `state_dict`, we call
 [`torch.load(f=PATH, weights_only=True)`](https://pytorch.org/docs/stable/generated/torch.load.html#torch.load) that loads an object saved using [`torch.save()`](https://pytorch.org/docs/stable/generated/torch.save.html#torch-save) from a file:

```
    model = TheModelClass(*args, **kwargs)
    model.load_state_dict(torch.load(PATH, weights_only=True))
    model.eval()
```

> 🔔 **Info**
+ Remember that you must call `model.eval()` before running inference
+ `f` - a file-like object or a string or `os.PathLike` object containing a file name. To work with paths, we'll use Python's [`pathlib`](https://docs.python.org/3/library/pathlib.html) module
+ Note that a `model` class must have been defined earlier, before calling [`model.load_state_dict()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.load_state_dict) on the object

In [ ]:
%%writefile -a helper_modules/utils.py

# function to load model from a specified path
def load_model(model: torch.nn.Module, path: pathlib.PosixPath):
  """
  Loads the model's state_dict from a specified path.

  Parameters
  ----------
  model : torch.nn.Module
      A new object of the model class.
  path : pathlib.PosixPath
      Path pointing to a previously saved model's state_dict.

  Returns
  -------
  model : torch.nn.Module
      The model returned after loading the state_dict.
  """
  # overwrite state_dict
  model.load_state_dict(torch.load(f=path, weights_only=True))


Appending to helper_modules/utils.py


## Make inference
> Here, I'll declare functions to make inference on `test` data
+ On a single random image
+ On multiple `(12)` random images

In [ ]:
%%writefile -a helper_modules/utils.py
# function to make inference on a single random image from testset
def make_single_inference(model: torch.nn.Module, dataset: torch.utils.data.Dataset,
                          label_map: dict, device: str):
  """
  Makes inference using a random data point from the test dataset.

  Parameters
  ----------
  model : torch.nn.Module
      A model (subclassing torch.nn.Module) to make inference.
  dataset : torch.utils.data.Dataset
      The Dataset to use for testing purposes.
  label_map : dict
      A dictionary mapping indices to labels (e.g., {0: 'O', 1: 'X'}).
  device : str
      Device on which to perform computation.

  Returns
  -------
  None
  """
  # get random image from test_set
  idx = np.random.choice(len(dataset))
  img, lb = dataset[idx]

  # make prediction
  with torch.inference_mode():
    model.to(device)  # move model to device
    model.eval()  # set eval mode
    lgts = model.to(device)(img.unsqueeze(0).to(device))
    pred = F.softmax(lgts, dim=1).argmax(dim=1)

  # print actual retrieved image
  plt.figure(figsize=(3.0, 3.0))
  # title with label
  if pred == lb:
    plt.title(
        f'Actual: {label_map[lb].title()}\nPred: {label_map[pred.item()].title()}',
        fontsize=8)
  else:  # if labels do not match, title = with red color
    plt.title(
        f'Actual: {label_map[lb].title()}\nPred: {label_map[pred.item()].title()}',
        fontsize=8, color='#de3163', weight='black')
  plt.axis(False)
  plt.imshow(img.permute(1,2,0))
  plt.show()

# function to make inference on 12 random images from testset
def make_multiple_inference(model: torch.nn.Module, dataset: torch.utils.data.Dataset,
                          label_map: dict, device: str):
  """
  Makes inference on multiple random images from the test dataset.

  Parameters
  ----------
  model : torch.nn.Module
      A model (subclassing torch.nn.Module) to make inference.
  dataset : torch.utils.data.Dataset
      The Dataset used for evaluation purposes.
  label_map : dict
      A dictionary mapping indices to labels (e.g., {0: 'O', 1: 'X'}).
  device : str
      Device on which to perform computation.

  Returns
  -------
  None
  """
  # get array of 12 random indices of images in test_dataset
  indices = np.random.choice(len(dataset), size=12, replace=False)
  # create subset from the 12 indices
  sub_set = Subset(dataset=dataset, indices=indices)

  # define a figure and subplots
  f, axs = plt.subplots(2, 6, figsize=(10, 8.5), layout='compressed')

  # move model to device & set eval mode
  model.to(device)
  model.eval()

  # loop through each subplot
  for i, ax in enumerate(axs.flat):
    img, lb = sub_set[i]  # return image and label

    # make inference on image returned
    with torch.inference_mode():
        lg = model(img.unsqueeze(0).to(device))
        pred = F.softmax(lg, dim=1).argmax(dim=1)

    ax.imshow(img.permute(1,2,0))
    ax.axis(False)
    if pred == lb:
        ax.set_title(
            f'Actual: {label_map[lb].title()}\nPred: {label_map[pred.item()].title()}',
            fontsize=7.5)
    else:  # if labels do not match, title = with red color
        ax.set_title(
            f'Actual: {label_map[lb].title()}\nPred: {label_map[pred.item()].title()}',
            fontsize=7.5, color='#de3163', weight='black')

  f.suptitle('Inference Made on 12 Random Test Images',
            # y=0.9,
            weight='black')
  plt.show()


Appending to helper_modules/utils.py


### Inference on custom images
> Here, I'll write the logic to make inference on custom image, where a user can upload an image of a playing card, and get prediction as well as top5 prediction probabilities

> First let's define image transforms for image used to run inference, and image plotted with the result

In [ ]:
%%writefile -a helper_modules/utils.py

# image transforms for inference
_img_transform = T.Compose(transforms=[
    T.PILToTensor(),
    T.ToDtype(dtype=torch.float32, scale=True),
    T.Resize((128,128)),
    T.CenterCrop((128, 128))
])

# image transform for plotting
_plot_transform = T.Compose([
    T.PILToTensor(),
    T.Resize((256, 256)),
    T.ToDtype(dtype=torch.float32, scale=True)
])


Appending to helper_modules/utils.py


#### Uploading the image
> Here, we run the following steps:
+ Create an `uploads` folder if it doesn't exist already
+ Upload an image using [`google.colab.file.upload()`](https://colab.research.google.com/notebooks/io.ipynb), and check if only ONE image is uploaded
+ Here, we use [`itertools.chain()`](https://docs.python.org/3/library/itertools.html#itertools.chain) function to combine multiple iterables gotten from [`pathlib.Path.glob()`](https://docs.python.org/3/library/pathlib.html#pathlib.Path.glob) into a single iterator
+ Get the image path
+ Turn image into a [`torch.Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor)

> 📝 **Note**
+ All path related functionality is implemeted using the `pathlib` module

In [ ]:
%%writefile -a helper_modules/utils.py

# function to upload an image for custom inference
def _upload_image() -> tuple[torch.Tensor, torch.Tensor]:
  """
    Handles image upload, validation, transformation, and cleanup.

    This function performs the following steps:
        - Ensures an upload directory exists.
        - Prompts the user to upload a single image file.
        - Validates that only one file is uploaded.
        - Applies image transformations for inference and plotting.
        - Deletes all uploaded images after processing.

    Returns
    -------
    tuple of torch.Tensor
        A tuple containing:
        - `inf_img`: The transformed image tensor used for inference.
        - `plot_img`: The transformed image tensor used for plotting or visualization.

    Raises
    ------
    Exception
        If more than one file is uploaded

    Notes
    -----
    Supported image formats include: .png, .jpg, .jpeg, and .gif.
    This function is intended to be used in a Colab notebook environment using `google.colab.files.upload`.
  """
  # create folder to upload, if not already there
  UPLOAD_DIR = Path('uploads')
  UPLOAD_DIR.mkdir(parents=True, exist_ok=True)

  # upload the file to the directory above
  upd = files.upload(target_dir=UPLOAD_DIR)

  # get all uploaded images
  iter_gen = chain(UPLOAD_DIR.glob('*.png'),
                    UPLOAD_DIR.glob('*.jpg'),
                    UPLOAD_DIR.glob('*.jpeg'),
                    UPLOAD_DIR.glob('*.gif'))

  if len(upd) != 1:
    # delete all uploaded images
    for img in iter_gen:
      img.unlink(missing_ok=True)
    # raise Exception
    raise Exception(f'Expected ONE image, but got {len(upd)}.\nRE-RUN the cell and upload a single image.')

  else:
    # get image path
    img_path = Path(next(iter(upd)))

    # turn image into tensor for inference
    inf_img = _img_transform(Image.open(img_path))
    # for plotting
    plot_img = _plot_transform(Image.open(img_path))

    # delete all uploads
    for img in iter_gen:
      img.unlink(missing_ok=True)

    return inf_img, plot_img


Appending to helper_modules/utils.py


#### Make prediction & plot
> Here, using the image tensor resulting from the step above, we make an inference, deriving the label and class probabilities
+ Also, we create a [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) of the top5 prediction probabilities and their corresponding class labels
+ For control over subplot placement in the final plot, we use [`matplotlib.gridspec.GridSpec`](https://matplotlib.org/stable/api/_as_gen/matplotlib.gridspec.GridSpec.html)
+ [`seaborn`](https://seaborn.pydata.org/index.html) is also used to make the [`barplot`](https://seaborn.pydata.org/generated/seaborn.barplot.html#seaborn-barplot)

In [ ]:
%%writefile -a helper_modules/utils.py

# function to perform infernce on a single image and display results
def custom_inference(model:torch.nn.Module, device:str, label_mapper:dict):
  """
    Performs inference on a single uploaded image, displays the prediction, and visualizes the top-5 class probabilities.

    This function:
        * Uploads a single image and applies necessary transformations.
        * Runs inference using the provided model and device.
        * Computes class probabilities using softmax.
        * Displays the top predicted label and its probability.
        * Creates and displays a bar plot of the top-5 predicted classes with their probabilities.

    Parameters
    ----------
    model : torch.nn.Module
        A trained PyTorch model for image classification.

    device : str
        The device to perform inference on (e.g., 'cpu' or 'cuda').

    label_mapper : dict
        A dictionary mapping class indices (int) to human-readable class labels (str).

    Raises
    ------
    Exception
        If more than one image is uploaded

    Notes
    -----
    - The image must be uploaded using an interactive Colab file upload widget.
    - The `_upload_image()` helper is responsible for transformation and validation.
    - Torch is used in inference mode to avoid tracking gradients.
    - This function assumes the model outputs raw logits for classification.

    Examples
    --------
    >>> label_map = {0: 'cat', 1: 'dog', 2: 'car'}
    >>> custom_inference(trained_model, device='cuda', label_mapper=label_map)
  """
  # get tensors from uploaded image
  inf_img, plot_img = _upload_image()

  #_____MAKE INFERENCE_____
  with torch.inference_mode():
    model.eval() # eval mode
    model.to(device)
    # make prediction
    logits = model(inf_img.unsqueeze(0).to(device))
    # scale logits on 0 -> scale
    logits = F.softmax(logits, dim=1)

    # label & class probability
    y_pred = logits.argmax(dim=1).item()
    class_prob = logits.max().item()

    #_____PROBABILITY THRESHOLD (Out Of Distribution Detection)_____
    if class_prob < 0.50:
      err_msg = 'The image uploaded is most likely NOT of a clear / valid PLAYING CARD\n\nRE-RUN THE CODE CELL AND UPLOAD A VALID PLAYING CARD IMAGE'
      raise Exception(err_msg)

    # get top5 indices & values from logits
    top5 = logits.topk(5)

  #_____MAKE DATAFRAME_____
  # make a dataframe out of the top5 predictions & probabilities
  df = pd.DataFrame({
      'Classes': [label_mapper[c.item()].title() for c in top5.indices.squeeze()],
      'Probabilities': top5.values.squeeze().cpu().numpy()
  })
  # sort dataframe in decsending order
  df.sort_values(by='Probabilities', ascending=False, inplace=True)

  #_____PLOT_____
  # set up figure, gridspec, axes
  f = plt.figure(figsize=(9, 3), layout='compressed')
  gs = GridSpec(figure=f, nrows=1, ncols=2, width_ratios=[1,3], wspace=0.05)
  ax1 = f.add_subplot(gs[0])
  ax2 = f.add_subplot(gs[1])

  # plot the card
  ax1.set_title(f'Predicted: {label_mapper[y_pred].title()}\nProbability: {class_prob:.2f}',
            fontsize=8.5, weight='black', color='#de3163')
  ax1.axis(False)
  ax1.imshow(plot_img.permute(1,2,0).clamp(min=0, max=1.0))

  # bar plot
  ax2.set_title('Top 5 Prediction Probabilities', weight='black', fontsize=10,
                color='#1f305e')
  ax2.set_ylabel('Classes', weight='black', fontsize=8.5)
  ax2.set_xlabel('Probabilities', weight='black', fontsize=8.5)
  ax2.tick_params(axis='both', labelsize=8)
  ax2.grid(axis='x', color='#dbd7d2')
  sns.barplot(
      data=df,
      x='Probabilities', y='Classes', hue='Classes', orient='h', palette='crest',
      legend=False, width=0.8, ax=ax2)
  # display
  plt.show()


Appending to helper_modules/utils.py


## Archive modules
> Here, we'll create a function to archive all the modules `*.py` files into a `*.zip` file with the help of [`make_archive`](https://docs.python.org/3/library/shutil.html#shutil.make_archive) function from the [`shutil`](https://docs.python.org/3/library/shutil.html#module-shutil) python module

> ✋ **Info**
+ The `zip` file containing the helper modules will be then uploaded to the GitHub repository [here](https://github.com/Martinmbiro/Card-classification/tree/main/helper%20modules). That way, the modules can be downloaded and extracted dynamically in code

In [ ]:
import shutil, pathlib

def archive_modules(path_to_files: pathlib.PosixPath, zip_name: str):
    """
    Archive a directory into a ZIP file.

    Parameters
    ----------
    path_to_files : pathlib.PosixPath
        The path to the directory or files to be archived.

    zip_name : str
        The name of the resulting ZIP file (without the extension).

    Returns
    -------
    None
        This function does not return any value. It creates a ZIP archive at the specified location.

    Notes
    -----
    This function uses `shutil.make_archive` to create the archive. The archive will be created
    in the current working directory unless a full path is provided in the `zip_name`.

    Examples
    --------
    >>> archive_modules(pathlib.Path('/path/to/files'), 'my_archive')
    This will create a ZIP archive named 'my_archive.zip' containing the files from the specified directory.

    """
    shutil.make_archive(
        base_name=zip_name, format='zip', root_dir=path_to_files)

In [ ]:
%%time
# archive modules
archive_modules(HELPER_MODULES, 'modules')

CPU times: user 828 µs, sys: 1.97 ms, total: 2.79 ms
Wall time: 4.26 ms


> ▶️ **Up Next**
+ Having created modules out of the most reusable code, I'll implement an end to tend project for classifying playing card images in the subsequent notebook, `02 Cards end to end.ipynb`